In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
inputs = [[1,2,3,2.5],
         [2.0,5.0,-1.0,2.0],
         [-1.5,2.7,3.3,-0.8]]

weights = [[0.2,0.8,-0.5,1.0],
          [0.5,-0.91,0.26,-0.5],
          [-0.26,-0.27,0.17,0.87]]

biases = [2,3,0.5]


layer_output = np.dot(inputs, np.array(weights).T) + biases


weights2 = [[0.2,0.8,-0.5],
          [0.5,-0.91,0.26],
          [-0.26,-0.27,0.17]]

biases2 = [2,3,0.5]

layer_output2 = np.dot(layer_output, np.array(weights2).T) + biases2

print(layer_output2)

[[ 2.7355   4.919   -0.66925]
 [ 2.232    9.1491  -1.2913 ]
 [ 3.1098   2.75535 -0.14595]]


In [3]:
X = [[1,2,3,2.5],
    [2.0,5.0,-1.0,2.0],
    [-1.5,2.7,3.3,-0.8]]

np.random.seed(0)

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights ) + self.biases
        return self.output

layer1 = Layer_Dense(4,5)
layer2 = Layer_Dense(5,6)

layer1 = layer1.forward(X)
layer2 = layer2.forward(layer1)
print(layer2)

[[-0.06635745 -0.07876527  0.10318969  0.11809888 -0.01409357  0.11115574]
 [-0.01154699 -0.04874198  0.00884775  0.07513459 -0.14256868  0.11875513]
 [ 0.13624801 -0.05087333  0.06957695  0.17383363 -0.04458113  0.03546162]]
